In [0]:
# pyspark functions
from pyspark.sql.functions import *
# URL processing
import urllib

In [0]:
# Define the path to the Delta table
delta_table_path = "dbfs:/user/hive/warehouse/authentication_credentials"

# Read the Delta table to a Spark DataFrame
aws_keys_df = spark.read.format("delta").load(delta_table_path)

In [0]:
# Get the AWS access key and secret key from the spark dataframe
ACCESS_KEY = aws_keys_df.select('Access key ID').collect()[0]['Access key ID']
SECRET_KEY = aws_keys_df.select('Secret access key').collect()[0]['Secret access key']
# Encode the secret key
ENCODED_SECRET_KEY = urllib.parse.quote(string=SECRET_KEY, safe="")

In [0]:
# Define S3 bucket details and mount point
AWS_S3_BUCKET = "user-0a1667ad2f7f-bucket"
MOUNT_NAME = "/mnt/0a1667ad2f7f/"
# Source url
SOURCE_URL = "s3n://{0}:{1}@{2}".format(ACCESS_KEY, ENCODED_SECRET_KEY, AWS_S3_BUCKET)
# Mount the drive
dbutils.fs.mount(SOURCE_URL, MOUNT_NAME)

--------------------------------------------------------------------------- 
 ExecutionError Traceback (most recent call last)
 <command-2166566293989627> in <module> 
 5 SOURCE_URL = "s3n://{0}:{1}@{2}" . format ( ACCESS_KEY , ENCODED_SECRET_KEY , AWS_S3_BUCKET ) 
 6 # Mount the drive 
 ----> 7 dbutils . fs . mount ( SOURCE_URL , MOUNT_NAME ) 

 /databricks/python_shell/dbruntime/dbutils.py in f_with_exception_handling (*args, **kwargs) 
 387 exc . __context__ = None 
 388 exc . __cause__ = None 
 --> 389 raise exc
 390 
 391 return f_with_exception_handling

 ExecutionError : An error occurred while calling o7477.mount.
: java.rmi.RemoteException: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/0a1667ad2f7f; nested exception is: 
	java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/0a1667ad2f7f
	at com.databricks.backend.daemon.data.client.DbfsClient.send0(DbfsClient.scala:128)
	at com.databricks.backend.daemon.data.client.DbfsClient.sendIdempotent(DbfsClient.scala:68)
	at com.databricks.backend.daemon.dbutils.DBUtilsCore.createOrUpdateMount(DBUtilsCore.scala:756)
	at com.databricks.backend.daemon.dbutils.DBUtilsCore.mount(DBUtilsCore.scala:776)
	at sun.reflect.GeneratedMethodAccessor3262.invoke(Unknown Source)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:750)
Caused by: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/0a1667ad2f7f
	at scala.Predef$.require(Predef.scala:281)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.$anonfun$insertMount$1(MetadataManager.scala:692)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.$anonfun$modifyAndVerify$2(MetadataManager.scala:1068)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.withRetries(MetadataManager.scala:841)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.modifyAndVerify(MetadataManager.scala:1057)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.insertMount(MetadataManager.scala:700)
	at com.databricks.backend.daemon.data.server.handler.MountHandler.receive(MountHandler.scala:128)
	at com.databricks.backend.daemon.data.server.session.SessionContext.$anonfun$queryHandlers$1(SessionContext.scala:54)
	at com.databricks.backend.daemon.data.server.session.SessionContext.$anonfun$queryHandlers$1$adapted(SessionContext.scala:53)
	at scala.collection.immutable.List.foreach(List.scala:431)
	at com.databricks.backend.daemon.data.server.session.SessionContext.queryHandlers(SessionContext.scala:53)
	at com.databricks.backend.daemon.data.server.DbfsServerBackend$$anonfun$receive$4.$anonfun$applyOrElse$9(DbfsServerBackend.scala:387)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:426)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.AttributionContext$.withValue(AttributionContext.scala:216)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:424)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:418)
	at com.databricks.rpc.ServerBackend.withAttributionContext(ServerBackend.scala:22)
	at com.databricks.logging.UsageLogging.withAttributionTags(UsageLogging.scala:472)
	at com.databricks.logging.UsageLogging.withAttributionTags$(UsageLogging.scala:455)
	at co

In [0]:
display(dbutils.fs.ls("/mnt/0a1667ad2f7f/topics"))

path,name,size,modificationTime
dbfs:/mnt/0a1667ad2f7f/topics/0a1667ad2f7f.geo/,0a1667ad2f7f.geo/,0,1707218816670
dbfs:/mnt/0a1667ad2f7f/topics/0a1667ad2f7f.pin/,0a1667ad2f7f.pin/,0,1707218816670
dbfs:/mnt/0a1667ad2f7f/topics/0a1667ad2f7f.user/,0a1667ad2f7f.user/,0,1707218816670


In [0]:
%sql
-- Disable format checks during the reading of Delta tables
SET spark.databricks.delta.formatCheck.enabled=false

key,value
spark.databricks.delta.formatCheck.enabled,false


In [0]:
df_pin = spark.read.json("/mnt/0a1667ad2f7f/topics/0a1667ad2f7f.pin/partition=0/")
df_geo = spark.read.json("/mnt/0a1667ad2f7f/topics/0a1667ad2f7f.geo/partition=0/")
df_user = spark.read.json("/mnt/0a1667ad2f7f/topics/0a1667ad2f7f.user/partition=0/")

country,ind,latitude,longitude,timestamp
Antarctica (the territory South of 60 deg S),2418,-88.4642,-171.061,2022-05-27T11:30:59
Antarctica (the territory South of 60 deg S),5162,-71.6607,-149.206,2019-09-27T19:06:43
Saint Vincent and the Grenadines,2301,13.4683,51.7244,2020-11-14T00:25:28
French Southern Territories,6014,-26.6026,155.206,2019-04-30T12:33:13
Cocos (Keeling) Islands,10794,-89.5236,-154.567,2022-01-01T02:26:50
Central African Republic,2074,-52.3213,-50.11,2019-11-03T05:41:59
British Virgin Islands,2293,-87.7946,-159.647,2022-03-21T10:46:53
Saint Kitts and Nevis,10663,-27.3474,-162.83,2019-07-25T18:53:51
Sao Tome and Principe,5293,-13.1463,-25.9649,2019-05-31T20:49:36
Sao Tome and Principe,9426,-29.8646,-78.1449,2021-06-30T02:14:47


In [0]:
# Unmount S3 bucket
# dbutils.fs.unmount("/mnt/0a1667ad2f7f/")

/mnt/0a1667ad2f7f/ has been unmounted.
Out[80]: True